In [ ]:
import os
import sys
import argparse

import torch.onnx
import torch
import random
import time
import shutil
from vision.ssd.vgg_ssd import create_vgg_ssd
from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd
from vision.ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite
from vision.ssd.squeezenet_ssd_lite import create_squeezenet_ssd_lite
from vision.ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite

from vision.ssd.config import mobilenetv1_ssd_config

In [ ]:
# Set the image file and sorted image storage address
imagefile_path = r"F:\\code\\detect_data"
# Save file address
savefile_path = r"F:\\code\\detect_data\\Annotations"

# Set the ratio of training set, verification set, and test set
trainval_rate = 0.8 # Training verification set
train_rate = 0.8 # training set


# Read the image dataset
total_images = os.listdir(imagefile_path)
# Record the total number of current image datasets
total_num = len(total_images)
image_list = range(total_num)

# Count the number of verification and test sets
validation_set_num = int(total_num * trainval_rate) # Number of training validation sets
train_set_num = int(validation_set_num * train_rate) # Number of training sets
print("Validation set size:", validation_set_num)
print("Train set size:", train_set_num)
 
 
# Random sampling as training set, test set and verification set
random.seed(1)  
validation_sample = random.sample(image_list, validation_set_num) 
train_sample = random.sample(validation_sample, train_set_num)  
 
# Data set partitioning
start = time.time()
test_num = 0 # test set
train_num = 0 # training set
val_num = 0 # Training verification set
 
for i in image_list:
    name = total_images[i]
    if i in validation_sample:
        if i in train_sample:
            directory = "train"
            train_num += 1
            images_path = os.path.join(os.getcwd(), "F:/code/detect_data/Annotations/{}".format(directory))
            if(not os.path.exists(images_path)):  
                # If the folder indicated by directory does not exist, create it with the following command
                os.mkdir(images_path)          
 
            file_path = os.path.join(imagefile_path, name)  
            newfile = os.path.join(savefile_path, os.path.join(directory,name))
            shutil.copyfile(file_path, newfile)           
 
        else:
            directory = "validation"
            images_path = os.path.join(os.getcwd(), "F:/code/detect_data/Annotations/{}".format(directory))
            if(not os.path.exists(images_path)):
                os.mkdir(images_path)                
 
            val_num += 1
            file_path = os.path.join(imagefile_path, name)
            newfile = os.path.join(savefile_path, os.path.join(directory,name))
            shutil.copyfile(file_path, newfile)
           
    else:
        directory = "test"
        images_path = os.path.join(os.getcwd(), "F:/code/detect_data/Annotations/{}".format(directory))
        if(not os.path.exists(images_path)):
            os.mkdir(images_path)
        
        test_num += 1 
        file_path = os.path.join(imagefile_path, name)
        newfile=os.path.join(savefile_path, os.path.join(directory,name))
        shutil.copyfile(file_path, newfile)
       
end = time.time() 
finish = end-start
print("Time taken:{0:.2f} seconds".format(finish))

In [ ]:
"""
Data set tag
labelImg image annotation tool was used to carry out feature annotation for Apple images of training set, test set and verification set respectively.
Create a VOC dataset
"""

In [ ]:
# Quickly convert the generated.xml file into a.csv file in batches
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
 
def xml_to_csv(path):  
    xml_list = []  
    for xml_file in glob.glob(path + '/*.xml'):  
        tree = ET.parse(xml_file)  
        root = tree.getroot()
       
        print(root.find('filename').text)  
        for member in root.findall('object'): 
            value = (root.find('filename').text[:-4],  
                int(root.find('size')[0].text),   #Width  
                int(root.find('size')[1].text),   #Height  
                member[0].text,   
                int(member[4][0].text), 
                int(float(member[4][1].text)), 
                int(member[4][2].text),  
                int(member[4][3].text)
                )  
            xml_list.append(value)
    column_name = ['ImageID', 'width', 'height', 'ClassName', 'XMin', 'YMin', 'XMax', 'YMax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)  
    return xml_df

In [ ]:
# Call the following store() function can directly carry out the above method call and the generated file.csv file storage
def store():
    for directory in ['train','test','validation']:
        xml_path = os.path.join(os.getcwd(), 'F:/code/detect_data/cooneo_xml/{}'.format(directory)) # Store.xml folder address
        xml_df = xml_to_csv(xml_path)
        xml_df.to_csv('F:/code/detect_data//Annotations/sub-{}-annotations-bbox.csv'.format(directory),
                      index=None) 
        print("Successfully converted xml to csv.")

In [ ]:
# Function call implementation
store()

In [ ]:
# Model training
""" Upload your own VOC dataset

unzip address -d Address for storing the decompressed file
unzip /content/drive/MyDrive/Cooneo/Apple-Rigeness/Annotations.zip -d  /content/drive/MyDrive/Cooneo/Apple-Rigeness/Apple-Rigeness

# View the current GPU environment configuration
nvidia-smi

from google.colab import drive
drive.mount('/content/drive')

Download pre-training models and training scripts
wget https://nvidia.box.com/shared/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth -O  /content/drive/MyDrive/Cooneo/models/mobilenet-v1-ssd-mp-0_675.pth
git clone https://github.com/dusty-nv/pytorch-ssd

Model training
# Train in a cloud environment
/usr/bin/python3 /content/pytorch-ssd/train_ssd.py --data=/content/drive/MyDrive/Cooneo/Apple-R
# Train in a local environment
python3 train_ssd.py -data= data set storage address --model-dir= trained model storage address --batch-size=10 --epochs=100

Model format conversion
# In a cloud environment
/usr/bin/python3 /content/pytorch-ssd/onnx_export.py  - input = / content/drive/MyDrive/Cooneo/Apple - Ripeness_model/mb1 - SSD - Epoch - 99 - Loss - 32774.71484375. PTH --model-dir=/content/drive/MyDrive/Cooneo/Apple-Ripeness_model
# Run locally
python3 model script address /onnx_export.py --input= Last trained model --model-dir= converted model storage address

Maturity detection using onnx_export.py
"" "